In [1]:
# importing libraries
from pyspark.sql import SparkSession
from pyspark.context import SparkContext
from pyspark.sql.types import *
from pyspark.sql.functions import *
from pyspark.sql.types import Row
from datetime import datetime

# Initializing Spark Session

In [2]:
spark = SparkSession.builder \
    .appName("Revenue By State ETL") \
    .config("spark.executor.memory", "4g") \
    .config("spark.executor.cores", "2") \
    .config("spark.driver.memory", "4g") \
    .config("spark.sql.shuffle.partitions", "200") \
    .config("spark.default.parallelism", "100") \
    .getOrCreate()

# Extracting

### Scraping file

In [64]:
import os
from scrap_file import FileHandler
scrap = FileHandler("C:\RevenueByState\data")

paths = os.listdir("C:\RevenueByState")
name_path = scrap.download_dir[scrap.download_dir.rfind('\\') + 1:]

if name_path not in paths:
    scrap.download_file()
elif "arrecadacao-estado.csv" not in os.listdir(scrap.download_dir):
    scrap.download_file()
else:
    print("Already exists!")


In [21]:
def num_partitions(file_size, num_of_files, spark):
    # Check the default partition size
    partition_size = int(spark.conf.get("spark.sql.files.maxPartitionBytes").replace("b",""))
    # Check the default open Cost in Bytes
    open_cost_size = int(spark.conf.get("spark.sql.files.openCostInBytes").replace("b",""))
    # Default parallelism
    parallelism = int(spark.sparkContext.defaultParallelism)
    # Total Actual File Size in Bytes
    total_file_size = file_size * num_of_files
    # Padded file size for Spark read
    padded_file_size = total_file_size + (num_of_files * open_cost_size)
    # Number of Bytes per Core
    bytes_per_core = padded_file_size / parallelism
    # Max Split Bytes
    max_bytes_per_split = min(partition_size, max(open_cost_size, bytes_per_core))
    # Total number of Partitions
    num_of_partitions = padded_file_size / max_bytes_per_split
    
    return num_of_partitions